In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import scipy.stats as ss
from scipy.optimize import curve_fit
from ntuple_lib import *
beautify_plots(plt)
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [ ]:
files = ['ntuples_condor/OutDir_{}/hist-Rel21sample.root'.format(i) for i in range(1, 10)]
truth_vtx_z = np.array([], dtype='object')
truth_vtx_z_HS = np.array([], dtype='object')

reco_vtx_z = np.array([], dtype='object')
reco_vtx_z_HS = np.array([], dtype='object')
reco_vtx_sum_pt2 = np.array([], dtype='object')

for file in files:
    root_file = uproot.open(file)
    for key in root_file.keys():
        if key.startswith('EventTree'):
            tree = root_file[key]
            event_truth_vtx_z = tree['truthvertex_z'].array(library='np')
            event_reco_vtx_z = tree['recovertex_z'].array(library='np')
            
            truth_vtx_z = np.concatenate((truth_vtx_z, event_truth_vtx_z))
            reco_vtx_z = np.concatenate((reco_vtx_z, event_reco_vtx_z))
            
            reco_vtx_sum_pt2 = np.concatenate((reco_vtx_sum_pt2, tree['recovertex_sumPt2'].array(library='np')))
            
            truth_vtx_z_HS = np.concatenate((truth_vtx_z_HS, event_truth_vtx_z[0]))
            reco_vtx_z_HS = np.concatenate((reco_vtx_z_HS, event_reco_vtx_z[0]))